# First  01-Getting and Cleaning Data.ipynb

In [ ]:
import os
import pandas as pd
os.chdir('/Users/Evan/DataScience/TB_Nation/Datasets/')
files = os.listdir()

In [ ]:
datasets = pd.DataFrame()
for i in range(1,len(files)):
    for j in os.listdir(files[i]):
        data = pd.read_excel(files[i]+'/'+ j ,skiprows=1).iloc[:1,0:5]
        data = data.rename(columns={'Unnamed: 0':'Area','发病数':'Incidence','死亡数':'Death','发病率':'Incidence_rate','死亡率':'Death_rate'})
        data['Year'],data['Month'],data['Day'] = files[i],j[4:6],'01'
        datasets = pd.concat([datasets,data])
# datasets.index=range(0,len(datasets))
datasets['Date'] = datasets['Year'] + datasets['Month'] + datasets['Day']
datasets.index = pd.to_datetime(datasets['Date'])
datasets = datasets.drop(['Day','Date'],axis=1)
datasets.head()

In [ ]:
datasets.to_excel('../Data/TB_nation.xlsx')

# First 02- Exploratory Data analysis.ipynb

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn-talk')
%config InlineBackend.figure_format = 'retina'

from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
datasets = pd.read_excel('../Data/TB_nation.xlsx',index_col='Date')

In [ ]:
datasets.head()

In [ ]:
datasets.Incidence_rate.plot(figsize=(12,8), title= 'Monthly TB Incidence Rate', fontsize=14)
# plt.savefig('month_TB.png', bbox_inches='tight')

In [ ]:
datasets_pred = datasets[datasets.index>='2014-01-1']
datasets = datasets[datasets.index<'2014-01-01']
datasets.shape

In [ ]:
decomposition = seasonal_decompose(datasets.Incidence_rate,freq=12)
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(12,6)

# First 03-SARIMA-First.ipynb

### First Difference 
#### Test of stationarity

In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import statsmodels.api as sm  
from statsmodels.tsa.stattools import acf,pacf

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
#     rolmean = pd.rolling_mean(timeseries, window=12)
    rolmean = timeseries.rolling(window=12,center=False).mean()
#     rolstd = pd.rolling_std(timeseries, window=12)
    rolstd = timeseries.rolling(window=12,center=False).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(10, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='BIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
datasets = pd.read_excel('../Data/TB_nation.xlsx',index_col='Date')
datasets_pred = datasets[datasets.index>='2014-01-1']
datasets = datasets[datasets.index<'2014-01-01']
datasets.shape

In [ ]:
datasets.head()

#### 原始序列的平稳性检验

In [ ]:
test_stationarity(datasets['Incidence_rate'])

#### First difference

In [ ]:
datasets['first_diff'] = datasets.Incidence_rate - datasets.Incidence_rate.shift(1)
test_stationarity(datasets.first_diff.dropna(inplace=False))

In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(datasets.first_diff.iloc[1:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(datasets.first_diff.iloc[1:], lags=40, ax=ax2)

#### Bulid Model

In [ ]:
mod = sm.tsa.SARIMAX(datasets.Incidence_rate, trend='n', order=(1,1,1), seasonal_order=(1,0,0,12))
results = mod.fit()
print(results.summary())

#### Plot

In [ ]:
datasets['forecast'] = results.predict(start = 1, end= 131, dynamic= False)  
datasets[['Incidence_rate', 'forecast']].plot(figsize=(12, 8))

In [ ]:
dta = pd.concat([datasets, datasets_pred])[['Death','Death_rate','Incidence','Incidence_rate','forecast','Year','Month']]
dta['forecast'] = results.predict(start=1,end=131,dynamic=False)
dta.tail(13)

In [ ]:
dta['forecast'] = results.predict(start = 2, end= 131, dynamic= False)  
dta[['Incidence_rate','forecast']].plot(figsize=(10, 8))

In [ ]:
dta['error_ARIMA'] = dta['forecast']-dta['Incidence_rate']
dta['error_ARIMA'].plot(figsize=(10,8))
sum(dta['error_ARIMA'].dropna())

In [ ]:
fig = plt.figure(figsize=(8,12))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dta.error_ARIMA.iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dta.error_ARIMA.iloc[13:], lags=40, ax=ax2)
# fig.savefig('Autocorrection.png',dpi=600)

In [ ]:
ta.to_excel('Difference.xlsx')